In [1]:
import os

def delete_zero_byte_files(folder_path):
    """
    어떤 폴더 내의 크기가 0KB인 파일을 일괄 삭제하는 함수
    """
    # 해당 디렉토리의 파일 목록을 가져옴
    files = os.listdir(folder_path)

    for file in files:
        file_path = os.path.join(folder_path, file)
        if os.path.getsize(file_path) == 0:
            os.remove(file_path)


def get_common_filename(folder_path):
    """
    어떤 폴더 내의 동일한 이름의 이미지 파일 & 텍스트 파일의 목록을 반환하는 함수
    """
    files = os.listdir(folder_path)

    # .jpg 파일과 .txt 파일을 각각 저장할 세트를 생성
    jpg_files = set()
    txt_files = set()

    for file in files:
        if file.endswith('.jpg'):
            jpg_files.add(file[:-4])
        elif file.endswith('.txt'):
            txt_files.add(file[:-4])
    
    # 동일한 이름을 도출
    return list(jpg_files & txt_files)


def cleanup_folder(folder_path):
    """
    YOLO 모델 학습 시 사용할 이미지 & 라벨데이터 정리하는 함수
    - 크기가 0KB일 시 삭제
    - 같은 이름의 이미지와 텍스트 파일이 없을 시, 즉 쌍이 아닌 파일은 삭제
    - 내부 기능을 다른 함수로 분할
    """
    # 해당 디렉토리의 파일 목록을 가져옴
    files = os.listdir(folder_path)

    # 파일 크기가 0KB라면 삭제
    delete_zero_byte_files(folder_path)

    # 동일한 이름을 도출
    common_files = get_common_filename(folder_path)

    # 동일한 파일명의 쌍이 없을 경우 삭제
    for file in files:
        filename, ext = os.path.splitext(file)  # 파일명, 확장자 분리
        if ext == '.jpg' and filename not in common_files:
            os.remove(os.path.join(folder_path, file))
        elif ext == '.txt' and filename not in common_files:
            os.remove(os.path.join(folder_path, file))


def cleanup_folder2(directory):
    """
    YOLO 모델 학습 시 사용할 이미지 & 라벨데이터 정리하는 함수
    - 크기가 0KB일 시 삭제
    - 같은 이름의 이미지와 텍스트 파일이 없을 시, 즉 쌍이 아닌 파일은 삭제
    """
    
    # 해당 디렉토리의 파일 목록을 가져옴
    files = os.listdir(directory)
    
    # .jpg 파일과 .txt 파일을 각각 저장할 세트를 생성
    jpg_files = set()
    txt_files = set()
    
    for file in files:
        file_path = os.path.join(directory, file)
        if os.path.getsize(file_path) == 0:
                os.remove(file_path)    # 파일 크기가 0KB라면 삭제
                continue
        if file.endswith('.jpg'):
            jpg_files.add(file[:-4])
        elif file.endswith('.txt'):
            txt_files.add(file[:-4])
    
    # 동일한 이름을 도출
    common_files = jpg_files & txt_files
    
    # 해당 디렉토리의 파일 목록을 가져옴
    files = os.listdir(directory)
    
    # 동일한 파일명의 쌍이 없을 경우 삭제
    for file in files:
        filename, ext = os.path.splitext(file)  # 파일명, 확장자 분리
        if ext == '.jpg' and filename not in common_files:
            os.remove(os.path.join(directory, file))
        elif ext == '.txt' and filename not in common_files:
            os.remove(os.path.join(directory, file))

In [2]:
### 학습 시 사용할 이미지 & 라벨 파일 정리
# cleanup_folder('./datasets/images/train')
cleanup_folder2('./datasets/whole_datasets')

In [3]:
import os

def change_into_one_label(folder_path):
    """
    각도에 따라 나뉜 지게차 라벨링을 하나로 통일하는 함수 (라벨 2, 3, 4 -> 라벨 2로 통일)
    """
    
    # 폴더 내의 모든 파일을 순회하며 작업
    for filename in os.listdir(folder_path):
        problem_flag = False    # txt 파일의 내용을 수정하기 전 문제가 없는지 확인
        
        if not filename.endswith('.txt'):
            continue    # txt 파일이 아니라면 건너뜀
        
        file_path = os.path.join(folder_path, filename)
        if os.path.getsize(file_path) == 0:
            continue    # 용량이 0이라면 건너뜀
        
        # 파일을 열고 각 줄을 읽는다.
        with open(file_path, "r") as f:
            lines = f.readlines()   # 전체 내용 저장
        
        # 각 줄을 처리하여 값을 변경
        modified_lines = []
        for line in lines:
            values = line.split()
            try:
                first_value = int(values[0])
            except Exception as e:
                problem_flag = True
                break   # 첫 값이 int형이 아니라면 취소
            
            # 첫 번째 값이 2 보다 큰 경우 2로 변경 (3, 4, ... -> 2)
            if first_value > 2:
                modified_line = "2" + line[1:]
            else:
                modified_line = line
            
            modified_lines.append(modified_line)
        
        if problem_flag:
            continue    # txt 파일 형식이 적절하지 않으면 건너뜀
        
        # 수정된 내용을 파일에 다시 저장
        with open(file_path, "w") as f:
            f.writelines(modified_lines)

In [5]:
### 지게차 라벨링 통일
change_into_one_label('./datasets/whole_datasets')

In [ ]:
def find_class_idx_error(folder_path):
    for filename in os.listdir(folder_path):
        if not filename.endswith('.txt'):
            continue
        
        file_path = os.path.join(folder_path, filename)

        # 파일을 열어 각 줄을 읽는다
        with open(file_path, "r") as f:
            for line in f:
                values = line.split()

                # 첫 번째 값(숫자)를 확인
                first_value = int(values[0])
                if first_value >= 5:
                    print(f'파일 {filename}의 {line}에서 클래스 값 {first_value}는 5 이상입니다.')

In [ ]:
# find_class_idx_error(folder_path)

In [6]:
### 정리된 파일들을 train, val 분리 & images, labels 수동으로 분리
import os
import random
import shutil

def train_valid_test_split(folder_path, ratio_train=0.8, ratio_valid=0.1, ratio_test=0.1):
    """
    폴더 내의 이미지 & 텍스트 파일 쌍을 임의의 비율대로 세 개의 폴더로 분리하는 함수 (비율 조절 가능)
    """

    files = os.listdir(folder_path) # 리스트

    # 파일 개수 계산 (이미지 & 텍스트 파일 쌍)
    common_files = get_common_filename(folder_path)
    num_files = len(common_files)

    # train, valid, test 폴더에 저장할 파일 개수 계산
    num_files_train = int(num_files * ratio_train)
    num_files_valid = int(num_files * ratio_valid)
    num_files_test = num_files - num_files_train - num_files_valid

    # 파일을 랜덤하게 섞기
    random.shuffle(common_files)

    # train, valid, test 폴더 생성
    if ratio_train:
        train_folder = os.path.join(folder_path, 'train')
        os.makedirs(train_folder, exist_ok=True)
    
    if ratio_valid:
        valid_folder = os.path.join(folder_path, 'valid')
        os.makedirs(valid_folder, exist_ok=True)
    
    if ratio_train:
        test_folder = os.path.join(folder_path, 'test')
        os.makedirs(test_folder, exist_ok=True)
    
    # 이미지 & 텍스트 파일쌍을 a, b, c 폴더로 이동
    for i in range(num_files_train):
        src_path = os.path.join(folder_path, common_files[i])
        dest_path = os.path.join(train_folder, common_files[i])
        shutil.move(src_path + '.jpg', dest_path + '.jpg')
        shutil.move(src_path + '.txt', dest_path + '.txt')

    for i in range(num_files_train, num_files_train + num_files_valid):
        src_path = os.path.join(folder_path, common_files[i])
        dest_path = os.path.join(valid_folder, common_files[i])
        shutil.move(src_path + '.jpg', dest_path + '.jpg')
        shutil.move(src_path + '.txt', dest_path + '.txt')
    
    for i in range(num_files_train + num_files_valid, num_files):
        src_path = os.path.join(folder_path, common_files[i])
        dest_path = os.path.join(test_folder, common_files[i])
        shutil.move(src_path + '.jpg', dest_path + '.jpg')
        shutil.move(src_path + '.txt', dest_path + '.txt')

In [7]:
# 훈련, 검증, 테스트 데이터셋으로 분리
train_valid_test_split('./datasets/whole_datasets', 0.8, 0.1, 0.1)